In [ ]:
# ======================
# Import, authentic, initialize earth engine library
# ======================
import ee
# ee.Authenticate() # only have to do once per week
ee.Initialize()

# ======================
# Import rest of libraries
# ======================
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime
import os
# import rasterio 
from glob import glob
import seaborn as sns
import folium 
import wrappers as wp # type: ignore (current workaround for pylance missing imports issue; doesn't affect functionality)


In [ ]:
# ======================
# Import landsat collections, shapefiles.
# Check for image collection updates once in a while
# ======================

LS5 = ee.ImageCollection("LANDSAT/LT05/C01/T1_SR")
LS7 = ee.ImageCollection("LANDSAT/LE07/C01/T1_SR")
LS8 = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR")

# NASA 30m SRTM
SRTM_30m = ee.Image("USGS/SRTMGL1_003")

# ======================
# Define region and points of interest
# ======================

# Unzen region of interest
test_aoi = ee.Geometry.Polygon(
        [[[130.27218028041096, 32.795519592838865],
          [130.27218028041096, 32.73027203877756],
          [130.36522074671956, 32.73027203877756],
          [130.36522074671956, 32.795519592838865]]])
# print(test_aoi.getInfo())

# intermediate; will change this later; point used to filter Landsat collections
filterpoint = ee.Geometry.Point([130.30344410868855, 32.761231395552514])

# ======================
# Parameters for each landsat collection
# ======================

parameters_LS5 = {'IMAGE_COLLECTION': LS5, 
            'FILTER_POINT': filterpoint,
            'LANDSAT_SAT': 'LS5',
            'CLOUD_COVER_OVER_LAND': 80,
            'IMAGE_QUALITY': 7,
            'GEOMETRIC_RMSE': 10,
}

parameters_LS7 = {'IMAGE_COLLECTION': LS7, 
            'FILTER_POINT': filterpoint,
            'LANDSAT_SAT': 'LS7', 
            'CLOUD_COVER_OVER_LAND': 80,
            'IMAGE_QUALITY': 7,
            'GEOMETRIC_RMSE': 10,
}

parameters_LS8 = {'IMAGE_COLLECTION': LS8, 
            'FILTER_POINT': filterpoint,
            'LANDSAT_SAT': 'LS8',
            'CLOUD_COVER_OVER_LAND': 80,
            'IMAGE_QUALITY': 7,
            'GEOMETRIC_RMSE': 10,
}

In [ ]:
# ======================
# Filter and harmonize
# ======================

LS5_filtered = wp.wrapper_prep(parameters_LS5)
LS7_filtered = wp.wrapper_prep(parameters_LS7)
LS8_filtered = wp.wrapper_prep(parameters_LS8)

# ======================
# Merge and get annual median composites of vegetation indices
# ======================

all_median_composites = wp.wrapper_VI(LS5_filtered, LS7_filtered, LS8_filtered)

In [ ]:
# ======================
# Export raster images out to external folder
# ======================

In [ ]:
# ======================
# Display GEE objects on map using folium
# folium map display code adapted from: https://colab.research.google.com/github/google/earthengine-api/blob/master/python/examples/ipynb/ee-api-colab-setup.ipynb#scrollTo=tRPULejJhBSl
# ======================

# Define a method for displaying Earth Engine image tiles to folium map
def add_ee_layer(self, ee_image_object, vis_params, name): 
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles = map_id_dict['tile_fetcher'].url_format,
        attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name = name,
        overlay = True,
        control = True).add_to(self)

# Add EE drawing method to folium
folium.Map.add_ee_layer = add_ee_layer

# Create a folium map object
# check lat/long positions for arguments
# Shimabara Peninsula: approximately [32.761231395552514, 130.30344410868855]
gee_map = folium.Map(location=[32.761231395552514, 130.30344410868855], zoom_start=10)

# Add image to map object
# gee_map.add_ee_layer(SRTM_30m.clip(Unzen_ROI_boundary), {}, 'SRTM')

# Add layer control
gee_map.add_child(folium.LayerControl())

# Display map
display(gee_map)